In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def calculate_probability(dc, bonus_min=0, bonus_max=0, advantage=False, dice_sides=20):
    calcfailprob = lambda b: max(1, dc - b - 1) / dice_sides
    factor = {True: 2, False: 1}.get(advantage, 1)
    prob_min = max(1 - calcfailprob(bonus_min) ** factor, 0)
    prob_max = max(1 - calcfailprob(bonus_max) ** factor, 0)
    return {"min": prob_min, "max": prob_max, "total": (prob_max + prob_min) / 2}

def plot_probability(dc_max, bonus_min, bonus_max, advantage, target):
    dc_values = range(1, dc_max + 1)
    probabilities = [
        calculate_probability(dc, bonus_min, bonus_max, advantage) for dc in dc_values
    ]

    min_probs = [p["min"] for p in probabilities]
    max_probs = [p["max"] for p in probabilities]
    total_probs = [p["total"] for p in probabilities]

    plt.figure(figsize=(10, 6))
    plt.plot(dc_values, min_probs, label="Min Prob", marker="o")
    plt.plot(dc_values, max_probs, label="Max Prob", marker="o")
    plt.plot(dc_values, total_probs, label="Average Prob", marker="o", linestyle="--")

    min_to_pass = max(target-bonus_min, 2)
    max_to_pass = max(target-bonus_max, 2)
    plt.axvline(min_to_pass, color="navy", linestyle="-.", label=f"Min to Pass ({min_to_pass})")
    plt.axvline(max_to_pass, color="red", linestyle="-.", label=f"Max to Pass ({max_to_pass})")
    plt.axvline(target, color="darkviolet", linestyle="-.", label=f"Target DC ({target})")

    plt.title("Probability of Success vs. Class Difficulty (DC)")
    plt.xlabel("Class Difficulty (DC)")
    plt.ylabel("Probability of Success")
    plt.xticks(np.arange(1, dc_max+1, 1))
    
    if advantage:
        plt.plot([], [], " ", label="Advantage")
        plt.yticks(np.arange(0, 1.05, 0.05))
    else:
        plt.yticks(np.arange(min(min_probs), 1.05, 0.05))
    
    plt.legend(loc="best")
    plt.grid(True)

bonus_min_slider = widgets.IntSlider(value=2, min=-1, max=13, description='Bonus Min:')
bonus_max_slider = widgets.IntSlider(value=10, min=-1, max=17, description='Bonus Max:')
dc_max_slider = widgets.IntSlider(value=20, min=20, max=35, description='DC Max:')
target_slider = widgets.IntSlider(value=15, min=1, max=35, description='Target DC:')
advantage_toggle = widgets.ToggleButtons(options=[('Off', False), ('On', True)], description='Advantage:')

output = widgets.Output()

def update_plot(change):
    with output:
        clear_output(wait=True)
        plot_probability(dc_max_slider.value, bonus_min_slider.value, bonus_max_slider.value, advantage_toggle.value, target_slider.value)
        plt.show()

bonus_min_slider.observe(update_plot, names='value')
bonus_max_slider.observe(update_plot, names='value')
dc_max_slider.observe(update_plot, names='value')
target_slider.observe(update_plot, names='value')
advantage_toggle.observe(update_plot, names='value')

vbox = widgets.VBox([dc_max_slider, target_slider, bonus_min_slider, bonus_max_slider, advantage_toggle, output])
display(vbox)
update_plot(None)
